In [1]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

import tensorflow as tf
import os,datetime

import pickle
from nltk.stem import WordNetLemmatizer
import string
import nltk
import re

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.preprocessing import OneHotEncoder

2024-11-03 19:20:53.868448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730683254.030162   26340 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730683254.060924   26340 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-03 19:20:54.361598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Take HTTP data as input!

In [2]:
df = pd.read_csv("../r4.2/http.csv",nrows=3000)
df['date'] = pd.to_datetime(df['date'])
df

,id,date,user,pc,url,content
0,{V1Y4-S2IR20QU-6154HFXJ},2010-01-02 06:55:16,LRR0148,PC-4275,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...
1,{Q5R1-T3EF87UE-2395RWZS},2010-01-02 07:00:13,NGF0157,PC-6056,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...
2,{X9O1-O0XW52VO-5806RPHG},2010-01-02 07:03:46,NGF0157,PC-6056,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...
3,{G5S8-U5OG04TE-5299CCTU},2010-01-02 07:05:26,IRM0931,PC-7188,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...
4,{L0R4-A9DH29VP-4553AUWM},2010-01-02 07:05:52,IRM0931,PC-7188,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...
...,...,...,...,...,...,...
2995,{Q6Z1-C5EQ59RR-6055BPNL},2010-01-02 15:08:29,RZC0746,PC-7500,http://nfl.com/Greece_runestones/dybeck/snzvyl...,needed 371 authority elevated daily 8th becaus...
2996,{M3N3-N2HQ40XO-3795GDSL},2010-01-02 15:08:34,NAH0503,PC-8267,http://vistaprint.com/Rosendale_trestle/rosend...,alert dangerous lifeguards affected inhg heavy...
2997,{D6N7-T8IV48CS-2221HLNT},2010-01-02 15:08:35,NWK0215,PC-8370,http://nfl.com/Greece_runestones/dybeck/snzvyl...,lauded generational 24 defended well reclaimed...
2998,{P1A4-K6ZK72FU-1547SSKT},2010-01-02 15:08:44,IRM0931,PC-7188,http://shareasale.com/1962_South_Vietnamese_In...,military accept museum when aground making rel...


## Extract features from Date!
The user logs are at different times. I divided the time into **4** different time frames.
- 0 = 12AM - 6AM
- 1 = 6AM - 12PM
- 2 = 12PM - 6PM
- 3 = 6PM - 12AM
Therefore, a new feature **time_frame** is made. Date is decomposed into 3 other numerical features: `day`,`month`, and `year`. Finally, date feature is dropped.

After dividng them into 4 different time frames in order to apply one-hot encoding.

**Why One-Hot?**
  - **No Ordinality:** Each time frame is represented independently without implying any order.
  - **Clarity:** Clearly distinguishes between different time frames.


In [80]:
# Define the function to categorize time frames
def categorize_time_frame(hour):
    if 0 <= hour < 6:
        return 0
    elif 6 <= hour < 12:
        return 1
    elif 12 <= hour < 18:
        return 2
    else:
        return 3

# Apply the function to create the 'time_frame' column
# new_df = df.copy()
df['time_frame'] = df['date'].dt.hour.apply(categorize_time_frame)
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

df=df.drop(columns="date")
df=df.drop(columns="id")

# Applying one-hot encoding

# Initialize OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
time_encoded = ohe.fit_transform(df[['time_frame']])

# Create DataFrame with One-Hot Encoded Columns
time_encoded_df = pd.DataFrame(time_encoded.astype(int), columns=ohe.get_feature_names_out(['time_frame']))

# Concatenate with Original DataFrame
df = pd.concat([df, time_encoded_df], axis=1).drop('time_frame', axis=1)

df

,user,pc,url,content,day,month,year,time_frame_1,time_frame_2,time_frame_3
0,LRR0148,PC-4275,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...,2,1,2010,1,0,0
1,NGF0157,PC-6056,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...,2,1,2010,1,0,0
2,NGF0157,PC-6056,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...,2,1,2010,1,0,0
3,IRM0931,PC-7188,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...,2,1,2010,1,0,0
4,IRM0931,PC-7188,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...,2,1,2010,1,0,0
...,...,...,...,...,...,...,...,...,...,...
99995,JIM0095,PC-9328,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...,5,1,2010,1,0,0
99996,BBS0039,PC-9436,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...,5,1,2010,1,0,0
99997,KAL0395,PC-0004,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...,5,1,2010,1,0,0
99998,LDB0090,PC-6824,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...,5,1,2010,1,0,0


# Label Encoding Year to reduce its size

In [81]:
le_year= LabelEncoder()
le_year.fit(df['year'])
df['year'] = le_year.transform(df['year'])
df

,user,pc,url,content,day,month,year,time_frame_1,time_frame_2,time_frame_3
0,LRR0148,PC-4275,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...,2,1,0,1,0,0
1,NGF0157,PC-6056,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...,2,1,0,1,0,0
2,NGF0157,PC-6056,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...,2,1,0,1,0,0
3,IRM0931,PC-7188,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...,2,1,0,1,0,0
4,IRM0931,PC-7188,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...,2,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
99995,JIM0095,PC-9328,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...,5,1,0,1,0,0
99996,BBS0039,PC-9436,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...,5,1,0,1,0,0
99997,KAL0395,PC-0004,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...,5,1,0,1,0,0
99998,LDB0090,PC-6824,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...,5,1,0,1,0,0


# Convert USER and PC based on binary encodings

In [82]:
user_label_encoder_path = "../encoded_objects/user_label_encoder.pkl"
user_binary_encoder_path = "../encoded_objects/user_binary_encoder.pkl"

pc_label_encoder_path = "../encoded_objects/pc_label_encoder.pkl"
pc_binary_encoder_path = "../encoded_objects/pc_binary_encoder.pkl"

label_encoded_df = df[['user','pc']]

with open(user_label_encoder_path, 'rb') as file:
    user_le = pickle.load(file)

label_encoded_df['user'] = user_le.transform(label_encoded_df.user)

with open(user_binary_encoder_path, 'rb') as file:
    user_binary_enc = pickle.load(file)

label_encoded_df = user_binary_enc.transform(label_encoded_df)

/tmp/ipykernel_34348/3161611282.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  label_encoded_df['user'] = user_le.transform(label_encoded_df.user)


In [83]:
with open(pc_label_encoder_path, 'rb') as file:
    pc_le = pickle.load(file)

label_encoded_df['pc'] = pc_le.transform(label_encoded_df.pc)

with open(pc_binary_encoder_path, 'rb') as file:
    pc_binary_enc = pickle.load(file)

label_encoded_df = pc_binary_enc.transform(label_encoded_df)

# At the end, combine the two encoded users and PC to the entire dataframe

df = pd.concat([df, label_encoded_df],axis=1)
df = df.drop(columns=['pc','user'])
df

,url,content,day,month,year,time_frame_1,time_frame_2,time_frame_3,user_0,user_1,...,pc_0,pc_1,pc_2,pc_3,pc_4,pc_5,pc_6,pc_7,pc_8,pc_9
0,http://msn.com/The_Human_Centipede_First_Seque...,remain representatives consensus concert altho...,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,festival off northwards than congestion partne...,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,long away reorganized baldwin seth business 18...,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,http://groupon.com/Leonhard_Euler/leonhard/tne...,among german schwein experimental becomes prev...,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,http://flickr.com/Inauguration_of_Barack_Obama...,kate criteria j 2008 highest 12 include books ...,2,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,http://city-data.com/No_Way_Out_2004/hotty/sre...,enlarged under generic advantage vision do any...,5,1,0,1,0,0,0,0,...,0,0,0,1,1,0,0,1,1,0
99996,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,an unknown afternoon dietary state law nationa...,5,1,0,1,0,0,0,0,...,0,0,1,0,1,1,0,1,0,1
99997,http://tigerdirect.com/European_Commission/bar...,begin top we band themselves harshly or fourth...,5,1,0,1,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
99998,http://microsoft.com/Meteorological_history_of...,until acquire flared get secondary minas sea u...,5,1,0,1,0,0,0,0,...,0,0,1,0,1,0,1,1,0,0


In [84]:
# Load the Word2Vec model
word2vec_model = KeyedVectors.load_word2vec_format('../models/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [85]:
# Download nltk data files (if not already installed)
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

In [86]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [87]:
def preprocess(text):
    # Lowercase the text and remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    # Remove stopwords and lemmatize
    cleaned_text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words)
    return cleaned_text

In [88]:
# Define function to get sentence vector
def get_sentence_vector(text):
    words = preprocess(text).split()  # Preprocess and split the text into words
    word_vectors = [word2vec_model[word] for word in words if word in word2vec_model]
    
    # If there are no valid words, return a zero vector
    if not word_vectors:
        return np.zeros(word2vec_model.vector_size)
    
    # Calculate the mean of word vectors for each sentence
    return np.mean(word_vectors, axis=0)

# Apply the function to your DataFrame
df['content_vector'] = df['content'].apply(get_sentence_vector)
df=df.drop(columns=['content'])
df

,url,day,month,year,time_frame_1,time_frame_2,time_frame_3,user_0,user_1,user_2,...,pc_1,pc_2,pc_3,pc_4,pc_5,pc_6,pc_7,pc_8,pc_9,content_vector
0,http://msn.com/The_Human_Centipede_First_Seque...,2,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"[0.025419906, 0.030988539, 0.0007039663, 0.048..."
1,http://urbanspoon.com/Plunketts_Creek_Loyalsoc...,2,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"[0.018577939, 0.038862318, -0.05105591, 0.0962..."
2,http://aa.com/Rhodocene/rhodocenium/fhaavatqrf...,2,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,"[-0.029726226, 0.024776326, -0.036115974, 0.05..."
3,http://groupon.com/Leonhard_Euler/leonhard/tne...,2,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0.0021921794, 0.045175172, 0.00957133, 0.1425..."
4,http://flickr.com/Inauguration_of_Barack_Obama...,2,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[-0.053494874, 0.025620727, 0.0054260255, 0.01..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,http://city-data.com/No_Way_Out_2004/hotty/sre...,5,1,0,1,0,0,0,0,0,...,0,0,1,1,0,0,1,1,0,"[0.002129632, 0.02529165, -0.014264081, 0.0252..."
99996,http://stubhub.com/Hoover_Dam/ickes/zbgbeplpyr...,5,1,0,1,0,0,0,0,1,...,0,1,0,1,1,0,1,0,1,"[-0.037862513, 0.056490157, 0.025104947, 0.064..."
99997,http://tigerdirect.com/European_Commission/bar...,5,1,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,"[0.06907099, -0.014849576, -0.023220062, 0.057..."
99998,http://microsoft.com/Meteorological_history_of...,5,1,0,1,0,0,0,0,1,...,0,1,0,1,0,1,1,0,0,"[0.04055214, 0.06912422, 0.0029149055, 0.08141..."


[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading wordnet: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading punkt: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>
[nltk_data] Error loading punkt_tab: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


False

In [4]:
df['url'] = df['url'].str.replace("_",' ')
df['url_tokens'] = df['url'].str.extractall(r'(\w+)')[0].groupby(level=0).agg(' '.join)

texts = df['url_tokens'].tolist()  # Convert the column of texts into a list
df

# Define a function for text preprocessing
def preprocess_text(text):
    # Tokenize
    tokens = word_tokenize(text.lower())
    # Remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return tokens

# Apply preprocessing to each row in the dataset
corpus = [preprocess_text(text) for text in texts]

# Create a dictionary and corpus from processed texts
dictionary = corpora.Dictionary(corpus)
bow_corpus = [dictionary.doc2bow(text) for text in corpus]

num_topics = list(range(5)[3:])
num_keywords = 15
LDA_models = {}
LDA_topics = {}

for i in num_topics:
    LDA_models[i] = LdaModel(corpus=bow_corpus,
                             id2word=dictionary,
                             num_topics=i,
                             update_every=1,
                             chunksize=len(bow_corpus),
                             passes=20,
                             alpha='auto',
                             random_state=42)

    shown_topics = LDA_models[i].show_topics(num_topics=i, 
                                             num_words=num_keywords,
                                             formatted=False)
    LDA_topics[i] = [[word[0] for word in topic[1]] for topic in shown_topics]

### Now create a function to derive the Jaccard similarity of two topics:

In [5]:
def jaccard_similarity(topic_1, topic_2):
    """
    Derives the Jaccard similarity of two topics

    Jaccard similarity:
    - A statistic used for comparing the similarity and diversity of sample sets
    - J(A,B) = (A ∩ B)/(A ∪ B)
    - Goal is low Jaccard scores for coverage of the diverse elements
    """
    intersection = set(topic_1).intersection(set(topic_2))
    union = set(topic_1).union(set(topic_2))
                    
    return float(len(intersection))/float(len(union))

### gensim has a built in model for topic coherence (this uses the 'c_v' option):

In [7]:
coherences = [CoherenceModel(model=LDA_models[i], texts=corpus, dictionary=dictionary, coherence='c_v').get_coherence()\
              for i in num_topics[:-1]]

### From here derive the ideal number of topics roughly through the difference between the coherence and stability per number of topics:

In [8]:
coh_sta_diffs = [coherences[i] - mean_stabilities[i] for i in range(num_keywords)[:-1]] # limit topic numbers to the number of keywords
coh_sta_max = max(coh_sta_diffs)
coh_sta_max_idxs = [i for i, j in enumerate(coh_sta_diffs) if j == coh_sta_max]
ideal_topic_num_index = coh_sta_max_idxs[0] # choose less topics in case there's more than one max
ideal_topic_num = num_topics[ideal_topic_num_index]

NameError: name 'mean_stabilities' is not defined

### Finally graph these metrics across the topic numbers:

Your ideal number of topics will maximize coherence and minimize the topic overlap based on Jaccard similarity. In this case it looks like we'd be safe choosing topic numbers around 14.

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.lineplot(x=num_topics[:-1], y=mean_stabilities, label='Average Topic Overlap')
ax = sns.lineplot(x=num_topics[:-1], y=coherences, label='Topic Coherence')

ax.axvline(x=ideal_topic_num, label='Ideal Number of Topics', color='black')
ax.axvspan(xmin=ideal_topic_num - 1, xmax=ideal_topic_num + 1, alpha=0.5, facecolor='grey')

y_max = max(max(mean_stabilities), max(coherences)) + (0.10 * max(max(mean_stabilities), max(coherences)))
ax.set_ylim([0, y_max])
ax.set_xlim([1, num_topics[-1]-1])
                
ax.axes.set_title('Model Metrics per Number of Topics', fontsize=25)
ax.set_ylabel('Metric Level', fontsize=20)
ax.set_xlabel('Number of Topics', fontsize=20)
plt.legend(fontsize=20)
plt.show()

In [ ]:
X_word2vec = np.array(df['content_vector'].tolist())  # Shape: (num_samples, 300)

# Assuming other features are already numeric, extract them as a NumPy array
X_other_features = df[['user', 'date']].values  # Adjust to your actual features

# Concatenate Word2Vec and other features along the last axis
X_combined = np.concatenate([X_word2vec, X_other_features], axis=1)
X_combined[0]